In [1]:
#testing google colab commit

In [2]:
!pip3 install duckduckgo-search newspaper4k lxml_html_clean

# !pip3 install --upgrade --quiet duckduckgo-search
# use this if rate limit is reached

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=72d6d893b7ae11e47158027c44ec0fe7d7dbab2ac1a275f51ab4952d789f51d8
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [3]:
import requests
import json

target_summaries_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/target.json"

# Fetch the JSON data from the URL
response = requests.get(target_summaries_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
target_summaries = json.loads(response.text)

# Now you can access the data within the target_summaries dictionary
print(target_summaries)  # Print the loaded JSON data

[{'query': 'Global economic outlook', 'summary': 'The global economic outlook for 2025 is marked by slowing growth, persistent inflation, and heightened uncertainty due to shifting trade policies and geopolitical tensions. According to the OECD, global GDP growth is projected to slow to 3.1% in 2025 and 3.0% in 2026, with the United States expected to grow at 2.2% in 2025 before decelerating, and the euro area at just 1.0%. China’s growth is also forecast to slow, reflecting broader weakness in domestic demand. Inflation remains above target in many economies, with G20 headline inflation projected at 3.8% in 2025. The World Bank notes that emerging markets and developing economies face weak investment, productivity challenges, and climate risks, with most low-income countries unlikely to reach middle-income status by 2050. Policy action is needed to foster stability, address debt, and accelerate sustainable development amid these headwinds.[2][9]'}, {'query': 'US presidential election 

In [4]:
import json
import time
from duckduckgo_search import DDGS
import requests
import os

def get_news_for_query(query, max_results=10):
    """Fetch news results for a single query"""
    try:
        ddgs = DDGS()
        results = list(ddgs.news(
            keywords=query,
            region="us-en",
            safesearch="off",
            timelimit="m",
            max_results=max_results
        ))

        # Extract just the URLs
        urls = [item['url'] for item in results]
        return urls

    except Exception as e:
        print(f"Error searching for '{query}': {str(e)}")
        return []

def main():
    # Detect if running in Colab
    in_colab = 'google.colab' in str(get_ipython())

    # Fetch the target summaries
    target_summaries_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/target.json"
    response = requests.get(target_summaries_url)
    response.raise_for_status()
    target_summaries = json.loads(response.text)

    # Dictionary to store query -> URLs mapping
    query_urls = {}

    # Process each query in the target summaries
    for item in target_summaries:
        query = item['query']
        print(f"\nProcessing query: {query}")

        # Get URLs for this query
        urls = get_news_for_query(query, max_results=10)
        query_urls[query] = urls

        print(f"Found {len(urls)} URLs for query: {query}")

        # Wait between different queries to avoid rate limiting
        if target_summaries.index(item) < len(target_summaries) - 1:  # Don't wait after the last query
            delay = 30  # 30 seconds between queries
            print(f"Waiting {delay} seconds before the next query...")
            time.sleep(delay)

    # Save the collected URLs to a JSON file
    output_file = 'news_urls.json'
    with open(output_file, 'w') as f:
        json.dump(query_urls, f, indent=2)

    print(f"\nAll URLs collected and saved to {output_file}")

    # If in Colab, provide code to download the file
    if in_colab:
        from google.colab import files
        files.download(output_file)
        print("\nFile download initiated. Check your browser's downloads.")

        # Also display the content for convenience
        print("\nJSON content:")
        print(json.dumps(query_urls, indent=2))

if __name__ == "__main__":
    main()


Processing query: Global economic outlook
Found 10 URLs for query: Global economic outlook
Waiting 30 seconds before the next query...


KeyboardInterrupt: 

In [ ]:
import newspaper

query_urls = {}

# Load the URLs from the JSON file
with open('news_urls.json', 'r') as f:
    query_urls = json.load(f)

In [ ]:
print(len(query_urls))
print(query_urls)

In [ ]:
import json
import time
import newspaper
import requests
from newspaper import Article
import os

def fetch_article_text(url):
    """Extract full text from a given news URL using newspaper"""
    try:
        article = Article(url)
        article.download()
        article.parse()

        return {
            'title': article.title,
            'text': article.text,
            'url': url
        }
    except Exception as e:
        print(f"Error fetching article from {url}: {str(e)}")
        return {
            'title': '',
            'text': '',
            'url': url,
            'error': str(e)
        }

def main():
    # Detect if running in Colab
    try:
        from google.colab import files
        in_colab = True
    except ImportError:
        in_colab = False

    # Load the URLs file
    try:
        with open('news_urls.json', 'r') as f:
            news_urls = json.load(f)
    except FileNotFoundError:
        print("news_urls.json not found. Please run the URL collection script first.")
        return

    # Dictionary to store all article data
    article_data = {}

    # Process each query
    for i, (query, urls) in enumerate(news_urls.items()):
        print(f"\nProcessing query {i+1}/{len(news_urls)}: {query}")
        print(f"Found {len(urls)} URLs to process")

        # Initialize array for this query's articles
        article_data[query] = []

        # Process each URL for this query
        for j, url in enumerate(urls):
            print(f"  Processing URL {j+1}/{len(urls)}: {url}")
            article_info = fetch_article_text(url)

            # Add to our data structure
            article_data[query].append(article_info)

            # Brief pause to be gentle on servers
            if j < len(urls) - 1:
                time.sleep(1)

        # Add a longer pause between queries
        if i < len(news_urls) - 1:
            print("Waiting 5 seconds before next query...")
            time.sleep(5)

    # Save to a new JSON file
    output_file = 'article_texts.json'
    with open(output_file, 'w') as f:
        json.dump(article_data, f, indent=2)

    print(f"\nAll article texts collected and saved to {output_file}")

    # If in Colab, provide download option
    if in_colab:
        files.download(output_file)
        print("\nFile download initiated. Check your browser's downloads.")

        # Display summary statistics
        total_articles = sum(len(articles) for articles in article_data.values())
        successful_articles = sum(1 for query_articles in article_data.values()
                                for article in query_articles if article['text'])

        print(f"\nSummary statistics:")
        print(f"Total queries processed: {len(article_data)}")
        print(f"Total articles attempted: {total_articles}")
        print(f"Successfully parsed articles: {successful_articles} ({successful_articles/total_articles*100:.1f}%)")

if __name__ == "__main__":
    main()

In [ ]:
from duckduckgo_search import DDGS
import newspaper


query = "applications of blockchain in healthcare"
# results = list(DDGS().text(query, max_results=10))

for result in results:
    print(result['title'])
    print(result['body'])

In [ ]:

article = newspaper.article('https://edition.cnn.com/2023/10/29/sport/nfl-week-8-how-to-watch-spt-intl/index.html')

print(article.authors)

In [ ]:
# news_result = DDGS().news(keywords="developments in climate change", region="us-en", safesearch="off", timelimit="m", max_results=10)
print(news_result)

In [ ]:
query = "applications of blockchain in healthcare"

# news_result = DDGS().news(
#     keywords=query,
#     region="us-en",
#     safesearch="off",
#     timelimit="m",
#     max_results=10)

# text_list = []

for result in news_result:
    try:
        article = newspaper.article(result['url'])
        article.download()
        article.parse()
        text_list.append({
            'title': result['title'],
            'body': result['body'],
            'url': result['url'],
            'full_text': article.text
        })
        print(article.title)
    except Exception as e:
        print(f"Skipping article due to error: {str(e)}")
        continue

In [ ]:
!pip3 install sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from transformers import pipeline


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
def generate_combined_summary(text_list, method='abstractive'):
    # Concatenate all articles with titles
    combined_text = ""
    for item in text_list:
        combined_text += f"{item['title']}\n{item['full_text']}\n\n"

    if method == 'extractive':
        # Using extractive summarization
        parser = PlaintextParser.from_string(combined_text, Tokenizer("english"))
        summarizer = LexRankSummarizer()
        summary = summarizer(parser.document, sentences_count=10)
        return " ".join([str(sentence) for sentence in summary])

    elif method == 'abstractive':
        # Using transformers for abstractive summarization
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        # Process in chunks if the combined text is too long
        max_chunk_length = 1024  # Approximate token limit for BART
        chunks = [combined_text[i:i+max_chunk_length] for i in range(0, len(combined_text), max_chunk_length)]

        summaries = []
        for chunk in chunks:
            if len(chunk) > 100:  # Ensure chunk is long enough to summarize
                summary = summarizer(chunk, max_length=150, min_length=50, do_sample=False)
                summaries.append(summary[0]['summary_text'])

        return " ".join(summaries)

In [ ]:
# Generate a single summary from all articles
overall_summary = generate_combined_summary(text_list)
print("Abstractive Overall Summary:")
print(overall_summary)

In [ ]:
overall_summary_extractive = generate_combined_summary(text_list, 'extractive')
print("Extractive Overall Summary:")
print(overall_summary_extractive)

In [ ]:
print(len(overall_summary_extractive))

In [ ]:
def topic_based_multi_doc_summary(text_list, num_sentences=15):
    # Extract all sentences from all documents
    all_sentences = []
    for item in text_list:
        sentences = sent_tokenize(item['full_text'])
        all_sentences.extend(sentences)

    # Calculate TF-IDF
    # Changed stop_words to a list instead of a set
    stop_words = stopwords.words('english')
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform(all_sentences)

    # Cluster sentences
    num_clusters = min(num_sentences, len(all_sentences))
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(tfidf_matrix)

    # Get closest sentences to cluster centers
    centers = kmeans.cluster_centers_
    ordered_sentences = []
    for i in range(num_clusters):
        # Find the sentence closest to this cluster center
        closest_idx = None
        closest_dist = float('inf')
        for j, sentence in enumerate(all_sentences):
            if kmeans.labels_[j] == i:
                # Convert tfidf_matrix[j] to dense array to match centers shape
                dist = ((tfidf_matrix[j].toarray() - centers[i]) ** 2).sum() ** 0.5
                if dist < closest_dist:
                    closest_dist = dist
                    closest_idx = j

        if closest_idx is not None:
            ordered_sentences.append((closest_idx, all_sentences[closest_idx]))

    # Sort by original order
    ordered_sentences.sort(key=lambda x: x[0])
    summary = " ".join([sentence for _, sentence in ordered_sentences])
    return summary

In [ ]:
# Generate topic-based summary
topic_summary = topic_based_multi_doc_summary(text_list)
print("Topic-based Summary:")
print(topic_summary)

In [ ]:
print(len(topic_summary))

In [ ]:
from collections import Counter
import re
import numpy as np

def weighted_multi_doc_summary(text_list, num_sentences=15):
    # Extract all text
    all_text = ""
    for item in text_list:
        all_text += item['full_text'] + " "

    # Find most common terms (potential key topics)
    words = re.findall(r'\w+', all_text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words and len(word) > 3]
    common_terms = [word for word, count in Counter(filtered_words).most_common(20)]

    # Score sentences based on common terms
    all_sentences = []
    for item in text_list:
        sentences = sent_tokenize(item['full_text'])
        for sentence in sentences:
            score = sum(1 for word in common_terms if word.lower() in sentence.lower())
            all_sentences.append((sentence, score))

    # Get top sentences
    all_sentences.sort(key=lambda x: x[1], reverse=True)
    top_sentences = [sentence for sentence, _ in all_sentences[:num_sentences]]

    # Order sentences as they appeared in original text
    sentence_positions = {}
    position = 0
    for item in text_list:
        sentences = sent_tokenize(item['full_text'])
        for sentence in sentences:
            if sentence in top_sentences:
                sentence_positions[sentence] = position
            position += 1

    ordered_top_sentences = sorted(top_sentences, key=lambda s: sentence_positions.get(s, float('inf')))
    summary = " ".join(ordered_top_sentences)
    return summary

In [ ]:
# Generate weighted summary
weighted_summary = weighted_multi_doc_summary(text_list)
print("Weighted Summary:")
print(weighted_summary)

In [ ]:
print(len(weighted_summary))

In [ ]:
print(len(text_list))
print(text_list[0])

In [ ]:
import time

# Function to get news with proper rate limiting
def get_rate_limited_news(keywords, delay_seconds=10, max_results=10, **kwargs):
    ddgs = DDGS()
    results = []

    # Set a smaller batch size to get fewer results per API call
    batch_size = 1
    remaining = max_results

    while remaining > 0:
        # Calculate how many results to request in this batch
        current_batch = min(batch_size, remaining)

        # Make the API call for just this batch
        batch_generator = ddgs.news(
            keywords=keywords,
            region="us-en",
            safesearch="off",
            timelimit="m",
            max_results=current_batch,
            **kwargs
        )

        # Process results from this batch
        batch_results = list(batch_generator)
        results.extend(batch_results)

        print(f"Fetched {len(batch_results)} articles (total: {len(results)}/{max_results})")

        # Update remaining count
        remaining -= len(batch_results)

        # Don't sleep after the last batch
        if remaining > 0:
            print(f"Waiting {delay_seconds} seconds before next request...")
            time.sleep(delay_seconds)

    return results



In [ ]:
# Use the rate-limited function
climate_news = get_rate_limited_news(
    keywords="developments in climate change",
    delay_seconds=10,
    max_results=10
)

# Now you have your results with proper rate limiting
print(f"\nTotal articles fetched: {len(climate_news)}")

In [ ]:
print(climate_news)

In [ ]:
climate_news_full_articles = []

for result in climate_news:
    try:
        article = newspaper.article(result['url'])
        article.download()
        article.parse()
        climate_news_full_articles.append({
            'title': result['title'],
            'body': result['body'],
            'url': result['url'],
            'full_text': article.text
        })
    except Exception as e:
        print(f"Skipping article due to error: {str(e)}")
        continue

In [ ]:
print(len(climate_news_full_articles))

In [ ]:
# Generate weighted summary
weighted_summary_climate = weighted_multi_doc_summary(climate_news_full_articles)
print("Weighted Summary Climate:")
print(weighted_summary_climate)

In [ ]:
# Generate topic-based summary
topic_summary_climate = topic_based_multi_doc_summary(climate_news_full_articles)
print("Topic-based Summary for Climate:")
print(topic_summary_climate)

In [ ]:
print(len(topic_summary_climate))

In [ ]:
# Generate a single summary from all articles
overall_summary = generate_combined_summary(climate_news_full_articles)
print("Abstractive Overall Summary:")
print(overall_summary)

In [ ]:
print(len(overall_summary))

In [ ]:
def compress_summary(summary, max_length=800):
    sentences = sent_tokenize(summary)

    # If already short enough, return as is
    if len(summary) <= max_length:
        return summary

    # Calculate importance score for each sentence using key phrase presence
    key_phrases = ["climate change", "global warming", "carbon emissions",
                  "renewable energy", "policy", "agreement", "temperature"]

    scored_sentences = []
    for sentence in sentences:
        # Score based on presence of key phrases and sentence position
        score = sum(2 if phrase in sentence.lower() else 0 for phrase in key_phrases)
        scored_sentences.append((sentence, score))

    # Sort by score and select top sentences
    scored_sentences.sort(key=lambda x: x[1], reverse=True)

    # Keep selecting sentences until we reach the length limit
    compressed = []
    current_length = 0
    for sentence, _ in scored_sentences:
        if current_length + len(sentence) <= max_length:
            compressed.append(sentence)
            current_length += len(sentence) + 1  # +1 for the space
        else:
            break

    return " ".join(compressed)

# Apply compression
compressed_topic_summary = compress_summary(overall_summary, max_length=600)
print(f"Original length: {len(overall_summary)}")
print(f"Compressed length: {len(compressed_topic_summary)}")

In [ ]:
print(compressed_topic_summary)

In [ ]:
def remove_redundancy(summary):
    sentences = sent_tokenize(summary)
    unique_sentences = []
    content_seen = set()

    for sentence in sentences:
        # Create a simplified representation by keeping only important words
        important_words = [w.lower() for w in word_tokenize(sentence)
                          if w.lower() not in stopwords.words('english') and len(w) > 3]

        # Skip sentences that are too similar to ones we've already kept
        sentence_content = " ".join(sorted(important_words))
        similarity_found = False

        for existing in content_seen:
            # Calculate Jaccard similarity
            common = len(set(sentence_content.split()) & set(existing.split()))
            total = len(set(sentence_content.split()) | set(existing.split()))
            if total > 0 and common/total > 0.6:  # More than 60% similar
                similarity_found = True
                break

        if not similarity_found:
            unique_sentences.append(sentence)
            content_seen.add(sentence_content)

    return " ".join(unique_sentences)

# Apply redundancy removal
non_redundant_summary = remove_redundancy(overall_summary)
print(f"After redundancy removal: {len(non_redundant_summary)}")

In [ ]:
def abstractive_compress(text, max_length=150, min_length=50):
    # Initialize summarizer
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    # If text is too long, break it into chunks and summarize each
    if len(text) > 1024:
        # Split into chunks of about 1000 characters
        chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
        summaries = []

        for chunk in chunks:
            if len(chunk) > 100:  # Only summarize substantial chunks
                summary = summarizer(chunk, max_length=max_length//len(chunks),
                                    min_length=min_length//len(chunks))
                summaries.append(summary[0]['summary_text'])

        return " ".join(summaries)
    else:
        # Summarize directly
        summary = summarizer(text, max_length=max_length, min_length=min_length)
        return summary[0]['summary_text']

# Apply abstractive compression to further reduce size
final_summary = abstractive_compress(topic_summary_climate, max_length=300, min_length=200)
print(f"Final summary length: {len(final_summary)}")

In [ ]:
print(final_summary)

In [ ]:
final_summary_2 = abstractive_compress(final_summary, max_length=30, min_length=10)
print(f"Final summary length: {len(final_summary_2)}")

In [ ]:
print(final_summary_2)

In [ ]:
import requests
import json

target_summaries_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/new_target.json"

# Fetch the JSON data from the URL
response = requests.get(target_summaries_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
target_data = json.loads(response.text)

# Create a dictionary mapping query to summary
target_summaries = {item['query']: item['summary'] for item in target_data}

# Display information about the loaded data
print(f"Loaded {len(target_summaries)} target summaries")
print("Sample target summary:")
first_key = list(target_summaries.keys())[0]
print(f"Query: {first_key}")
print(f"Summary: {target_summaries[first_key][:200]}...")

In [ ]:
news_urls_json_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/news_urls.json"
response = requests.get(news_urls_json_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
news_urls = json.loads(response.text)
print(f"Loaded URLs for {len(news_urls)} topics")
print("Sample topic with URLs:")
first_key = list(news_urls.keys())[0]
print(f"Topic: {first_key}")
print(f"URLs: {news_urls[first_key]}")

In [ ]:
article_texts_json_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/article_texts.json"
response = requests.get(article_texts_json_url)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Load the JSON data into a Python dictionary
article_texts = json.loads(response.text)
print(f"Loaded article texts for {len(article_texts)} topics")
topics_with_articles = sum(1 for articles in article_texts.values() if articles)
print(f"Number of topics with at least one article: {topics_with_articles}")

In [14]:
import nltk
# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

True

In [ ]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

# Function to extract first 150 words from text
def extract_first_150_words(text):
    """Extract the first 150 words from an article."""
    if not text or not isinstance(text, str) or not text.strip():
        return ""
    words = word_tokenize(text)
    first_150 = ' '.join(words[:150])
    return first_150

# Generate summaries from the first 150 words of the first article for each query
generated_summaries = {}
for query, articles in article_texts.items():
    if articles and len(articles) > 0:
        # Find the first article with actual text
        article_text = ""
        for article in articles:
            if isinstance(article, dict) and 'text' in article and article['text']:
                article_text = article['text']
                break

        if article_text:
            generated_summaries[query] = extract_first_150_words(article_text)
        else:
            print(f"No valid text found for query: {query}")
    else:
        print(f"No articles found for query: {query}")

print(f"Generated {len(generated_summaries)} summaries")
# Show an example generated summary
if generated_summaries:
    example_query = list(generated_summaries.keys())[0]
    print(f"Example generated summary for '{example_query}':\n{generated_summaries[example_query][:200]}...")

In [ ]:
def print_queries_and_summaries(summaries_dict):
    """
    Print all queries and their summaries in a nice format.

    Args:
        summaries_dict (dict): Dictionary mapping queries to their summaries
    """
    print("\n===== QUERIES AND SUMMARIES =====\n")

    for query, summary in summaries_dict.items():
        print(f"QUERY: {query}")
        print(f"SUMMARY: {summary}")
        print("-" * 80)  # Separator line

# Example usage
if generated_summaries:
    print_queries_and_summaries(generated_summaries)

In [7]:
!pip install lexrank

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 10.5 MB/s eta 0:00:00


In [10]:
import json
import requests
import numpy as np
from nltk.tokenize import sent_tokenize
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

True

In [12]:
# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

True

In [15]:
def lexrank_summarize(sentences, summary_length=150):
    """
    Implement LexRank algorithm to extract the most salient sentences.

    Args:
        sentences: List of sentences
        summary_length: Approximate target length in words

    Returns:
        A string containing the summary
    """
    if not sentences:
        return ""

    # Create TF-IDF matrix
    vectorizer = TfidfVectorizer(stop_words='english')
    try:
        tfidf_matrix = vectorizer.fit_transform(sentences)
    except ValueError:  # If we get an empty matrix or other error
        return sentences[0] if sentences else ""

    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Apply LexRank to find sentence scores
    scores = np.sum(similarity_matrix, axis=1)

    # Rank sentences by score and select them until we reach word limit
    ranked_sentences = [(score, i, s) for i, (score, s) in enumerate(zip(scores, sentences))]
    ranked_sentences.sort(reverse=True)  # Sort by score (descending)

    # Select sentences for summary
    selected_sentences = []
    word_count = 0

    for _, idx, sentence in ranked_sentences:
        words_in_sentence = len(sentence.split())
        if word_count + words_in_sentence <= summary_length:
            selected_sentences.append((idx, sentence))
            word_count += words_in_sentence
        else:
            break

    # Order the selected sentences by their original position
    selected_sentences.sort(key=lambda x: x[0])

    # Join sentences to form a summary
    summary = ' '.join(sentence for _, sentence in selected_sentences)
    return summary

# Generate summaries for each topic
topic_summaries = {}

for topic, articles in article_texts.items():
    print(f"Processing topic: {topic}")

    # Collect all non-empty text from articles
    all_text = ""
    for article in articles:
        if isinstance(article, dict) and 'text' in article and article['text']:
            all_text += article['text'] + " "

    if all_text:
        # Tokenize into sentences
        sentences = sent_tokenize(all_text)

        # Generate LexRank summary
        summary = lexrank_summarize(sentences)
        topic_summaries[topic] = summary
    else:
        topic_summaries[topic] = "Insufficient text available for summarization."

# Print the summaries
for topic, summary in topic_summaries.items():
    print(f"\n## {topic}\n")
    print(summary)
    print("-" * 80)

Processing topic: Global economic outlook
Processing topic: US presidential election updates
Processing topic: Climate change and extreme weather events
Processing topic: Advances in artificial intelligence
Processing topic: Major tech company earnings reports
Processing topic: International conflict and peace negotiations
Processing topic: COVID-19 pandemic developments
Processing topic: Renewable energy breakthroughs
Processing topic: Stock market trends and analysis
Processing topic: Major sports event highlights
Processing topic: Space exploration and discoveries
Processing topic: Cybersecurity threats and data breaches
Processing topic: Healthcare policy changes
Processing topic: Immigration and border policy news
Processing topic: Education reform and school closures
Processing topic: Natural disasters and humanitarian response
Processing topic: Major legal cases and court rulings
Processing topic: Celebrity news and entertainment industry updates
Processing topic: Scientific res

In [16]:
import json
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from collections import Counter
import re
import requests

# Load article texts
article_texts_json_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/article_texts.json"
response = requests.get(article_texts_json_url)
response.raise_for_status()
article_texts = json.loads(response.text)

print(f"Loaded article texts for {len(article_texts)} topics")

# Define functions for all three summarization methods
def lexrank_summarize(sentences, summary_length=150):
    """
    Implement LexRank algorithm to extract the most salient sentences.
    """
    if not sentences:
        return ""

    # Create TF-IDF matrix
    vectorizer = TfidfVectorizer(stop_words='english')
    try:
        tfidf_matrix = vectorizer.fit_transform(sentences)
    except ValueError:  # If we get an empty matrix or other error
        return sentences[0] if sentences else ""

    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Apply LexRank to find sentence scores
    scores = np.sum(similarity_matrix, axis=1)

    # Rank sentences by score and select them until we reach word limit
    ranked_sentences = [(score, i, s) for i, (score, s) in enumerate(zip(scores, sentences))]
    ranked_sentences.sort(reverse=True)  # Sort by score (descending)

    # Select sentences for summary
    selected_sentences = []
    word_count = 0

    for _, idx, sentence in ranked_sentences:
        words_in_sentence = len(sentence.split())
        if word_count + words_in_sentence <= summary_length:
            selected_sentences.append((idx, sentence))
            word_count += words_in_sentence
        else:
            break

    # Order the selected sentences by their original position
    selected_sentences.sort(key=lambda x: x[0])

    # Join sentences to form a summary
    summary = ' '.join(sentence for _, sentence in selected_sentences)
    return summary

def topic_based_multi_doc_summary(texts, summary_length=150):
    """
    Generate a topic-based summary using sentence clustering
    """
    # Extract all sentences from all texts
    all_sentences = []
    for text in texts:
        if text:
            sentences = sent_tokenize(text)
            all_sentences.extend(sentences)

    if not all_sentences:
        return ""

    # Calculate TF-IDF
    stop_words = stopwords.words('english')
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    try:
        tfidf_matrix = vectorizer.fit_transform(all_sentences)
    except ValueError:
        return all_sentences[0] if all_sentences else ""

    # Determine number of clusters (sentences to extract)
    num_clusters = min(10, len(all_sentences))  # Adjust as needed to get ~150 words

    # Cluster sentences
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(tfidf_matrix)

    # Get closest sentences to cluster centers
    centers = kmeans.cluster_centers_
    ordered_sentences = []

    for i in range(num_clusters):
        # Find the sentence closest to this cluster center
        closest_idx = None
        closest_dist = float('inf')

        for j, sentence in enumerate(all_sentences):
            if kmeans.labels_[j] == i:
                # Calculate distance to center
                dist = ((tfidf_matrix[j].toarray() - centers[i]) ** 2).sum() ** 0.5
                if dist < closest_dist:
                    closest_dist = dist
                    closest_idx = j

        if closest_idx is not None:
            ordered_sentences.append((closest_idx, all_sentences[closest_idx]))

    # Sort by original order
    ordered_sentences.sort(key=lambda x: x[0])

    # Take sentences until we reach ~150 words
    summary_sentences = []
    word_count = 0

    for _, sentence in ordered_sentences:
        words_in_sentence = len(sentence.split())
        if word_count + words_in_sentence <= summary_length:
            summary_sentences.append(sentence)
            word_count += words_in_sentence
        else:
            break

    summary = ' '.join(summary_sentences)
    return summary

def weighted_multi_doc_summary(texts, summary_length=150):
    """
    Generate a summary based on term frequency and common terms
    """
    if not texts or all(not text for text in texts):
        return ""

    # Combine all text
    all_text = " ".join([text for text in texts if text])

    # Find most common terms (potential key topics)
    words = re.findall(r'\w+', all_text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words and len(word) > 3]
    common_terms = [word for word, count in Counter(filtered_words).most_common(20)]

    # Score sentences based on common terms
    all_sentences = []
    for text in texts:
        if text:
            sentences = sent_tokenize(text)
            for sentence in sentences:
                score = sum(1 for word in common_terms if word.lower() in sentence.lower())
                all_sentences.append((sentence, score))

    # Get top sentences
    all_sentences.sort(key=lambda x: x[1], reverse=True)

    # Select sentences until we reach word limit
    top_sentences = []
    word_count = 0

    for sentence, _ in all_sentences:
        words_in_sentence = len(sentence.split())
        if word_count + words_in_sentence <= summary_length:
            top_sentences.append(sentence)
            word_count += words_in_sentence
        else:
            break

    # Order sentences for readability (by position in original text)
    sentence_positions = {}
    position = 0

    for text in texts:
        if text:
            sentences = sent_tokenize(text)
            for sentence in sentences:
                if sentence in top_sentences:
                    sentence_positions[sentence] = position
                position += 1

    ordered_top_sentences = sorted(top_sentences, key=lambda s: sentence_positions.get(s, float('inf')))
    summary = ' '.join(ordered_top_sentences)
    return summary

# Generate and print summaries for each topic with all three methods
for topic, articles in article_texts.items():
    print(f"\n\n## {topic}")
    print("=" * 80)

    # Collect all non-empty text from articles
    texts = []
    for article in articles:
        if isinstance(article, dict) and 'text' in article and article['text']:
            texts.append(article['text'])

    if not texts:
        print("No text content available for this topic.")
        continue

    # Method 1: LexRank
    print("\n1. LexRank Summary:")
    print("-" * 40)
    all_sentences = []
    for text in texts:
        sentences = sent_tokenize(text)
        all_sentences.extend(sentences)

    lexrank_summary = lexrank_summarize(all_sentences)
    print(lexrank_summary)

    # Method 2: Topic-Based
    print("\n2. Topic-Based Summary:")
    print("-" * 40)
    topic_summary = topic_based_multi_doc_summary(texts)
    print(topic_summary)

    # Method 3: Weighted
    print("\n3. Weighted Summary:")
    print("-" * 40)
    weighted_summary = weighted_multi_doc_summary(texts)
    print(weighted_summary)

    print("\n" + "=" * 80)

Loaded article texts for 30 topics


## Global economic outlook

1. LexRank Summary:
----------------------------------------
Today's average refinance rates

30-year fixed-rate 15-year fixed-rate 30-year fixed-rate jumbo 5/1 ARM 10-year fixed-rate 30-year fixed-rate refinance 15-year fixed-rate refinance 10-year fixed refinance

Today's average mortgage rates on Apr. 30-year fixed-rate refinance

The average 30-year fixed refinance rate right now is 6.82%, an increase of 8 basis points compared to one week ago. 15-year fixed-rate refinance

For 15-year fixed refinances, the average rate is currently at 6.10%, an increase of 4 basis points compared to one week ago. Also, 15-year refinance rates are typically lower than 30-year refinance rates, which will help you save more in the long run. 10-year fixed-rate refinance

The average 10-year fixed refinance rate right now is 6.02%, a decrease of 1 basis points compared to one week ago.

2. Topic-Based Summary:
----------------------------

In [17]:
def semantic_summarization(texts, summary_length=150):
    """
    Generate a summary based on semantic similarity between sentences using
    sentence embeddings to capture underlying meaning.

    Args:
        texts: List of article texts
        summary_length: Approximate target length in words

    Returns:
        A string containing the summary based on semantic meaning
    """
    # Import necessary libraries
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np
    from nltk.tokenize import sent_tokenize

    if not texts or all(not text for text in texts):
        return ""

    # Extract all sentences
    all_sentences = []
    for text in texts:
        if text:
            sentences = sent_tokenize(text)
            all_sentences.extend(sentences)

    if not all_sentences:
        return ""

    # Load pre-trained sentence embedding model
    try:
        model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight model with good performance

        # Generate embeddings for all sentences
        embeddings = model.encode(all_sentences)

        # Calculate centroid (average) of all sentence embeddings to find the "semantic center"
        centroid = np.mean(embeddings, axis=0)

        # Calculate semantic similarity between each sentence and the centroid
        similarities = cosine_similarity([centroid], embeddings)[0]

        # Rank sentences by semantic similarity to the centroid
        ranked_sentences = [(sim, i, sent) for i, (sim, sent) in enumerate(zip(similarities, all_sentences))]
        ranked_sentences.sort(reverse=True)  # Sort by similarity (descending)

        # Select sentences for summary (keeping track of word count)
        selected_sentences = []
        word_count = 0

        for _, idx, sentence in ranked_sentences:
            words_in_sentence = len(sentence.split())
            if word_count + words_in_sentence <= summary_length:
                selected_sentences.append((idx, sentence))
                word_count += words_in_sentence
            else:
                break

        # Order the selected sentences by their original position for readability
        selected_sentences.sort(key=lambda x: x[0])

        # Join sentences to form a summary
        summary = ' '.join(sentence for _, sentence in selected_sentences)
        return summary

    except ImportError:
        print("Please install sentence-transformers: pip install sentence-transformers")
        # Fallback to LexRank if sentence-transformers is not available
        return lexrank_summarize(all_sentences, summary_length)

In [18]:
import json
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
import requests
from tqdm import tqdm

# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Load article texts
article_texts_json_url = "https://raw.githubusercontent.com/hspgit/IR-Project-Text-summarization/main/article_texts.json"
response = requests.get(article_texts_json_url)
response.raise_for_status()
article_texts = json.loads(response.text)

print(f"Loaded article texts for {len(article_texts)} topics")

# Install sentence-transformers if needed
!pip install -q sentence-transformers

# Generate summaries with all four methods
lexrank_summaries = []
topic_based_summaries = []
weighted_summaries = []
semantic_summaries = []

# Process each topic
for query, articles in tqdm(article_texts.items(), desc="Generating summaries"):
    # Collect all non-empty text from articles
    texts = []
    for article in articles:
        if isinstance(article, dict) and 'text' in article and article['text']:
            texts.append(article['text'])

    if not texts:
        continue

    # Extract all sentences for LexRank
    all_sentences = []
    for text in texts:
        sentences = sent_tokenize(text)
        all_sentences.extend(sentences)

    # Generate summaries with each method
    lexrank_summary = lexrank_summarize(all_sentences)
    topic_summary = topic_based_multi_doc_summary(texts)
    weighted_summary = weighted_multi_doc_summary(texts)
    semantic_summary = semantic_summarization(texts)

    # Add to respective lists
    lexrank_summaries.append({"query": query, "summary": lexrank_summary})
    topic_based_summaries.append({"query": query, "summary": topic_summary})
    weighted_summaries.append({"query": query, "summary": weighted_summary})
    semantic_summaries.append({"query": query, "summary": semantic_summary})

# Save summaries to JSON files
with open('lexrank_summaries.json', 'w') as f:
    json.dump(lexrank_summaries, f, indent=4)

with open('topic_based_summaries.json', 'w') as f:
    json.dump(topic_based_summaries, f, indent=4)

with open('weighted_summaries.json', 'w') as f:
    json.dump(weighted_summaries, f, indent=4)

with open('semantic_summaries.json', 'w') as f:
    json.dump(semantic_summaries, f, indent=4)

print("All summaries generated and saved to JSON files")

Loaded article texts for 30 topics
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.0 MB/s eta 0:00:00


Generating summaries:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating summaries: 100%|██████████| 30/30 [02:44<00:00,  5.49s/it]

All summaries generated and saved to JSON files


In [19]:
def hybrid_ensemble_summarization(texts, summary_length=150):
    """
    Generate a summary using a hybrid ensemble approach that combines:
    1. LexRank for centrality
    2. Semantic similarity for meaning
    3. Topic clustering for coverage
    4. Weighted term importance for key concepts

    The ensemble weights the sentences from each method and selects the most
    representative sentences while removing redundancy.

    Args:
        texts: List of article texts
        summary_length: Approximate target length in words

    Returns:
        A string containing the hybrid ensemble summary
    """
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np
    from nltk.tokenize import sent_tokenize
    from collections import Counter
    import re

    if not texts or all(not text for text in texts):
        return ""

    # Extract all sentences
    all_sentences = []
    for text in texts:
        if text:
            sentences = sent_tokenize(text)
            all_sentences.extend(sentences)

    if not all_sentences:
        return ""

    # 1. LexRank scores
    try:
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(all_sentences)
        similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
        lexrank_scores = np.sum(similarity_matrix, axis=1)

        # Normalize lexrank scores to 0-1 range
        if np.max(lexrank_scores) > 0:
            lexrank_scores = lexrank_scores / np.max(lexrank_scores)
    except:
        lexrank_scores = np.zeros(len(all_sentences))

    # 2. Semantic similarity scores
    try:
        model = SentenceTransformer('all-MiniLM-L6-v2')
        embeddings = model.encode(all_sentences)
        centroid = np.mean(embeddings, axis=0)
        semantic_scores = cosine_similarity([centroid], embeddings)[0]
    except:
        semantic_scores = np.zeros(len(all_sentences))

    # 3. Topic clustering scores (based on distance to cluster centers)
    try:
        # Use fewer clusters for short texts
        num_clusters = min(5, len(all_sentences))
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        kmeans.fit(tfidf_matrix)

        # Calculate distance to closest cluster center
        cluster_scores = np.zeros(len(all_sentences))
        for i, sentence_idx in enumerate(range(len(all_sentences))):
            cluster_id = kmeans.labels_[sentence_idx]
            dist = np.linalg.norm(tfidf_matrix[sentence_idx].toarray() - kmeans.cluster_centers_[cluster_id])
            # Convert distance to similarity (closer = higher score)
            cluster_scores[i] = 1.0 / (1.0 + dist)
    except:
        cluster_scores = np.zeros(len(all_sentences))

    # 4. Key term importance scores
    try:
        all_text = " ".join(texts)
        words = re.findall(r'\w+', all_text.lower())
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word not in stop_words and len(word) > 3]
        common_terms = [word for word, _ in Counter(filtered_words).most_common(20)]

        term_scores = np.zeros(len(all_sentences))
        for i, sentence in enumerate(all_sentences):
            term_scores[i] = sum(1 for term in common_terms if term.lower() in sentence.lower())

        # Normalize term scores
        if np.max(term_scores) > 0:
            term_scores = term_scores / np.max(term_scores)
    except:
        term_scores = np.zeros(len(all_sentences))

    # 5. Position importance (sentences early in documents often contain key info)
    position_scores = np.zeros(len(all_sentences))
    sent_pos = 0
    doc_lengths = []

    # Calculate document lengths first
    for text in texts:
        if text:
            doc_lengths.append(len(sent_tokenize(text)))

    # Now calculate position scores
    doc_idx = 0
    sent_pos = 0
    for text in texts:
        if text:
            sentences = sent_tokenize(text)
            for i, _ in enumerate(sentences):
                idx = all_sentences.index(sentences[i])
                # Give higher scores to sentences that appear early in documents
                # and slightly boost the first sentence of each document
                if i == 0:
                    position_bonus = 0.5  # Bonus for being the first sentence
                else:
                    position_bonus = 0

                position_scores[idx] = 1.0 - (i / doc_lengths[doc_idx]) + position_bonus
            doc_idx += 1

    # Combine all scores with weights
    combined_scores = (
        0.25 * lexrank_scores +
        0.25 * semantic_scores +
        0.2 * cluster_scores +
        0.2 * term_scores +
        0.1 * position_scores
    )

    # Rank sentences by combined score
    ranked_sentences = [(score, i, sent) for i, (score, sent) in enumerate(zip(combined_scores, all_sentences))]
    ranked_sentences.sort(reverse=True)  # Sort by score (descending)

    # Select sentences while avoiding redundancy
    selected_indices = []
    selected_sentences = []
    word_count = 0

    # We'll use sentence embeddings to check for redundancy
    try:
        selected_embeddings = []

        for _, idx, sentence in ranked_sentences:
            # Skip if we've reached the word limit
            words_in_sentence = len(sentence.split())
            if word_count + words_in_sentence > summary_length:
                continue

            # Check for redundancy using semantic similarity
            if selected_embeddings:
                sent_embedding = embeddings[idx].reshape(1, -1)
                similarities = cosine_similarity(sent_embedding, np.array(selected_embeddings))
                max_similarity = np.max(similarities)

                # Skip if too similar to already selected sentences
                if max_similarity > 0.7:  # Threshold for similarity
                    continue

            # Add this sentence
            selected_indices.append(idx)
            selected_sentences.append(sentence)
            word_count += words_in_sentence

            if selected_embeddings == []:
                selected_embeddings = [embeddings[idx]]
            else:
                selected_embeddings.append(embeddings[idx])

            if word_count >= summary_length:
                break
    except:
        # Fallback if semantic redundancy check fails
        for _, idx, sentence in ranked_sentences:
            words_in_sentence = len(sentence.split())
            if word_count + words_in_sentence <= summary_length:
                selected_indices.append(idx)
                selected_sentences.append(sentence)
                word_count += words_in_sentence
            if word_count >= summary_length:
                break

    # Order the selected sentences by their original position
    ordered_selections = sorted(zip(selected_indices, selected_sentences), key=lambda x: x[0])
    summary = ' '.join(sentence for _, sentence in ordered_selections)

    return summary

In [20]:
# Generate summaries with the hybrid ensemble method
hybrid_summaries = []

# Process each topic
for query, articles in tqdm(article_texts.items(), desc="Generating hybrid summaries"):
    # Collect all non-empty text from articles
    texts = []
    for article in articles:
        if isinstance(article, dict) and 'text' in article and article['text']:
            texts.append(article['text'])

    if not texts:
        continue

    # Generate hybrid ensemble summary
    hybrid_summary = hybrid_ensemble_summarization(texts)
    hybrid_summaries.append({"query": query, "summary": hybrid_summary})

# Save summaries to JSON file
with open('hybrid_summaries.json', 'w') as f:
    json.dump(hybrid_summaries, f, indent=4)

print("Hybrid summaries generated and saved to 'hybrid_summaries.json'")

Generating hybrid summaries: 100%|██████████| 30/30 [02:16<00:00,  4.54s/it]

Hybrid summaries generated and saved to 'hybrid_summaries.json'
